In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from urllib import parse
import time
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [ ]:
#chrome driver가 설치될 파일 경로 확인
import os
os.getcwd()

In [ ]:
#크롤링할 사이트 불러오기
driver = webdriver.Chrome()
driver.maximize_window()
driver.get('https://taas.koroad.or.kr/gis/mcm/mcl/initMap.do?menuId=GIS_GMP_STS_RSN')
time.sleep(1)

In [ ]:
#사고년도 설정 (2021년)
year_start=driver.find_element(By.XPATH, '//*[@id="ptsRafYearStart"]/option[3]')
year_start.click()
year_end=driver.find_element(By.XPATH, '//*[@id="ptsRafYearEnd"]/option[3]')
year_end.click()

In [ ]:
#지역(제주도) 설정
region=driver.find_element(By.XPATH, '//*[@id="ptsRafSido"]/option[17]')
region.click()

In [ ]:
#제주도 지역 전체 설정
select=driver.find_element(By.XPATH, '//*[@id="ptsRafSigungu"]/option[1]')
select.click()

In [ ]:
#모든 사고 유형 선택
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[2]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[3]/input')
acc.click()
acc=driver.find_element(By.XPATH, '//*[@id="ptsRafCh1AccidentContent"]/li[4]/input')
acc.click()

In [ ]:
#사고부문 선택 (어린이 관련 사고로 한정)
opt=driver.find_element(By.XPATH,'//*[@id="ptsRafSimpleCondition"]/option[16]')
opt.click()

In [ ]:
#검색
driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[2]/p/a').send_keys(Keys.ENTER)

In [ ]:
#건수 확인
cnt=(driver.find_element(By.XPATH, '//*[@id="regionAccidentFind"]/div[3]/div[1]/span').text)
cnt=int(tn.replace(",",""))

In [ ]:
#크롤링 함수 정의
#coord_ls는 좌표계, srs_ls는 좌표계 종류를 의미하는 리스트
def crawling(num):
    coord_ls = []
    srs_ls = []
    for i in tqdm(range(0,num)):
        driver.execute_script(f'gis.srh.msh.selectListUi({i}, 11);') # 개별 사고지점
        time.sleep(1) # 로드
        bbox = driver.find_elements(By.CLASS_NAME,'olTileImage')[-1]
        src = parse.unquote(bbox.get_attribute('src')) # get src and unquote from 16byte
        src_split = src.split('&')
        bbox_text = src_split[-3][5:]
        coords = [float(i) for i in bbox_text.split(',')]
        x, y = (coords[0]+coords[2])/2, (coords[1]+coords[3])/2 # center position
        srs = src_split[-4][4:]
        coord_ls.append([x, y])
        srs_ls.append(srs)
    return coord_ls, srs_ls

In [ ]:
#크롤링 진행
coord,srs=crawling(cnt)
coord=np.array(coord)

In [ ]:
#좌표 변환 함수
import pyproj
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5181
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fy, fx])[0]

In [ ]:
# 좌표계 정보 설정
p1_type = "epsg:5181"
p2_type = "epsg:4326"

# project_array() 함수 실행
result = project_array(coord, p1_type, p2_type)

In [ ]:
df=pd.DataFrame(result,columns=['위도','경도'])

In [ ]:
from PyKakao import Local
api = Local(service_key = "카카오API에서 발급받은 키")

In [ ]:
#사고지점 정보를 기반으로 행정동 정보 추출
df['행정동']=None
for i in range(0,len(df)):
    hd=api.geo_coord2regioncode(df.iloc[i,1],df.iloc[i,0],dataframe=True)
    df.iloc[i,2]=hd.iloc[1,5]

In [ ]:
#전처리 결과 데이터셋 생성
df=pd.concat([final_df,df],axis=0)
df.to_csv('사고 정보.csv',index=False,encoding='cp949')